In [2]:
import pandas as pd
file_path = 'train.json'
df = pd.read_json(file_path)
df_train = df.T.reset_index(drop=True)
print(df_train)

                                               original  \
0            What do unstable isotope studies indicate?   
1     What is the basic unit of territorial division...   
2               Which genus lack tentacles and sheaths?   
3     Long-lived memory cells can remember previous ...   
4     What led to Newcastle's rise to power as milit...   
...                                                 ...   
7177         What does the beroe do when pursuing prey?   
7178  A static equilibrium between three sources is ...   
7179                   When did Li Tan reject a revolt?   
7180  What was there a significant minority of in Wo...   
7181   How long after a banquet with Tugh Temur did ...   

                                              disfluent  
0     What do petrologists no what do unstable isoto...  
1     What is the second level of territorial divisi...  
2     Juvenile platyctenids no wow Which genus lack ...  
3     When a pathogen is met again scratch that I me...  
4

In [3]:
import pandas as pd
file_path = 'test.json'
df = pd.read_json(file_path)
df_test = df.T.reset_index(drop=True)
print(df_test)

                                               original  \
0                  In what country is Normandy located?   
1                    When were the Normans in Normandy?   
2         From which countries did the Norse originate?   
3                             Who was the Norse leader?   
4     What century did the Normans first gain their ...   
...                                                 ...   
3638  Amazon rain forest experienced another mild dr...   
3639  The Amazon releases how much carbon dioxide ea...   
3640  The 2010 drought had three what were vegetatio...   
3641  In 2005 the force absorbed how much carbon dio...   
3642            In 2010 the force absorbed 8 Gt of what   

                                              disfluent  
0     In what country is Norse found no wait Normand...  
1     From which countries no tell me when were the ...  
2     From which Norse leader I mean countries did t...  
3                 When I mean Who was the Norse leader?  
4

In [4]:
import pandas as pd
file_path = 'dev.json'
df_valid = pd.read_json(file_path)
df_valid = df.T.reset_index(drop=True)
print(df_valid)

                                               original  \
0                  In what country is Normandy located?   
1                    When were the Normans in Normandy?   
2         From which countries did the Norse originate?   
3                             Who was the Norse leader?   
4     What century did the Normans first gain their ...   
...                                                 ...   
3638  Amazon rain forest experienced another mild dr...   
3639  The Amazon releases how much carbon dioxide ea...   
3640  The 2010 drought had three what were vegetatio...   
3641  In 2005 the force absorbed how much carbon dio...   
3642            In 2010 the force absorbed 8 Gt of what   

                                              disfluent  
0     In what country is Norse found no wait Normand...  
1     From which countries no tell me when were the ...  
2     From which Norse leader I mean countries did t...  
3                 When I mean Who was the Norse leader?  
4

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [5]:
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Convert the training and test DataFrames to Hugging Face Datasets
dataset_train = Dataset.from_pandas(df_train)
dataset_valid = Dataset.from_pandas(df_valid)

# Load the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Tokenize the input data
def tokenize_function(examples):
    model_inputs = tokenizer(examples['disfluent'], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples['original'], max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the training and test datasets
tokenized_train = dataset_train.map(tokenize_function, batched=True)
tokenized_valid = dataset_valid.map(tokenize_function, batched=True)

# Fine-tuning setup with test dataset for evaluation
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=1000,
    save_total_limit=2,
    evaluation_strategy="steps",  # Evaluate during training
    eval_steps=500,  # Evaluation frequency
)

# Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,  # Provide the test dataset for evaluation
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine-tuned-bart')
tokenizer.save_pretrained('./fine-tuned-bart')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

...

Map:   0%|          | 0/3643 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.638100,0.010122
1000,0.010900,0.009451
1500,0.009600,0.009391
2000,0.007800,0.008813
2500,0.006000,0.009086
3000,0.006100,0.008694
3500,0.005200,0.008798
4000,0.004000,0.008932
4500,0.003800,0.008615
5000,0.003600,0.008700


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./fine-tuned-bart/tokenizer_config.json',
 './fine-tuned-bart/special_tokens_map.json',
 './fine-tuned-bart/vocab.json',
 './fine-tuned-bart/merges.txt',
 './fine-tuned-bart/added_tokens.json')

In [6]:
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer
from datasets import Dataset, load_metric
import numpy as np
import torch

# Convert the validation DataFrame to a Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_test)

# Load the fine-tuned model and tokenizer
fine_tuned_model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')
fine_tuned_tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')

# Move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fine_tuned_model.to(device)

# Create a pipeline with the fine-tuned model
bart_model_fine_tuned = pipeline('text2text-generation', model=fine_tuned_model, tokenizer=fine_tuned_tokenizer, device=0)

# Tokenize the validation dataset using the same function as during training
def tokenize_function(examples):
    # Tokenize the inputs (disfluent text)
    model_inputs = fine_tuned_tokenizer(examples['disfluent'], max_length=512, truncation=True, padding="max_length")

    # Tokenize the labels (original text) using `text_target`
    labels = fine_tuned_tokenizer(text_target=examples['original'], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_test = dataset_test.map(tokenize_function, batched=True)

# Evaluate the model on the validation set
predictions = []
references = []

for example in tokenized_valid:
    # Generate output using the fine-tuned model
    output = bart_model_fine_tuned(example['disfluent'], max_length=50, min_length=5, do_sample=False)
    predictions.append(output[0]['generated_text'])
    references.append(example['original'])

# Calculate BLEU score
bleu_metric = load_metric("bleu")
bleu_score = bleu_metric.compute(predictions=[pred.split() for pred in predictions],
                                 references=[[ref.split()] for ref in references])
print(f"BLEU score: {bleu_score['bleu']}")

# Calculate Exact Match Accuracy
exact_match = np.mean([pred == ref for pred, ref in zip(predictions, references)])
print(f"Exact Match Accuracy: {exact_match * 100:.2f}%")


Map:   0%|          | 0/3643 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
<ipython-input-6-3ae9b06c1e43>:44: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("bleu")


The repository for bleu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bleu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
BLEU score: 0.887961983804101
Exact Match Accuracy: 68.49%


In [14]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback, get_linear_schedule_with_warmup # Make sure to import EarlyStoppingCallback
from datasets import Dataset
# Load the tokenizer and model for T5
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')
def tokenize_function(examples):
    model_inputs = tokenizer(examples['disfluent'], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(text_target=examples['original'], max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_train = dataset_train.map(tokenize_function, batched=True)
tokenized_valid = dataset_valid.map(tokenize_function, batched=True)
# Define training arguments with regularization and learning rate scheduler
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,                    # Adjusted number of epochs to avoid overfitting
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,                      # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                     # L2 regularization (weight decay)
    logging_dir='./logs',
    logging_steps=200,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Create an instance of EarlyStoppingCallback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,  # Provide the validation dataset for evaluation
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback]
)

# Fine-tune the model
trainer.train()


scheduler = get_linear_schedule_with_warmup(
    trainer.optimizer,
    num_warmup_steps=500,
    num_training_steps=trainer.args.num_train_epochs * len(df_train) // training_args.per_device_train_batch_size
)

model.save_pretrained('./fine-tuned-t5')
tokenizer.save_pretrained('./fine-tuned-t5')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/7182 [00:00<?, ? examples/s]

Map:   0%|          | 0/3643 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.037300,0.011687
1000,0.008400,0.008011
1500,0.007300,0.007500
2000,0.006100,0.007229
2500,0.005500,0.007044
3000,0.005400,0.007040
3500,0.005700,0.006980


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


('./fine-tuned-t5/tokenizer_config.json',
 './fine-tuned-t5/special_tokens_map.json',
 './fine-tuned-t5/spiece.model',
 './fine-tuned-t5/added_tokens.json')

In [15]:
from transformers import pipeline, T5ForConditionalGeneration, T5Tokenizer
from datasets import Dataset, load_metric
import numpy as np
import torch

# Convert the validation DataFrame to a Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_test)

# Load the fine-tuned T5 model and tokenizer
fine_tuned_model = T5ForConditionalGeneration.from_pretrained('./fine-tuned-t5')
fine_tuned_tokenizer = T5Tokenizer.from_pretrained('./fine-tuned-t5')

# Move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fine_tuned_model.to(device)

# Create a pipeline with the fine-tuned model
t5_model_fine_tuned = pipeline('text2text-generation', model=fine_tuned_model, tokenizer=fine_tuned_tokenizer, device=0)

# Tokenize the validation dataset using the same function as during training
def tokenize_function(examples):
    # Tokenize the inputs (disfluent text)
    model_inputs = fine_tuned_tokenizer(examples['disfluent'], max_length=512, truncation=True, padding="max_length")

    # Tokenize the labels (original text) using `text_target`
    labels = fine_tuned_tokenizer(text_target=examples['original'], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_test = dataset_test.map(tokenize_function, batched=True)

# Evaluate the model on the validation set
predictions = []
references = []

for example in tokenized_test:
    # Generate output using the fine-tuned model
    input_text = "translate English to English: " + example['disfluent']  # T5 expects a task prefix
    output = t5_model_fine_tuned(input_text, max_length=50, min_length=5, do_sample=False)
    predictions.append(output[0]['generated_text'])
    references.append(example['original'])

# Calculate BLEU score
bleu_metric = load_metric("bleu")
bleu_score = bleu_metric.compute(predictions=[pred.split() for pred in predictions],
                                 references=[[ref.split()] for ref in references])
print(f"BLEU score: {bleu_score['bleu']}")

# Calculate Exact Match Accuracy
exact_match = np.mean([pred == ref for pred, ref in zip(predictions, references)])
print(f"Exact Match Accuracy: {exact_match * 100:.2f}%")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/3643 [00:00<?, ? examples/s]

BLEU score: 0.7326330557119155
Exact Match Accuracy: 45.24%


In [31]:
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback, get_linear_schedule_with_warmup
from datasets import Dataset

# Define model and tokenizer
model_name = 'facebook/bart-base'  # Changed to BART Base
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

# Define training arguments with regularization and learning rate scheduler
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,                    # Number of epochs
    per_device_train_batch_size=8,        # Increased batch size to balance performance
    per_device_eval_batch_size=8,
    warmup_steps=500,                      # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                     # L2 regularization (weight decay)
    logging_dir='./logs',
    logging_steps=200,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Implement Early Stopping Callback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback]
)

# Training the model
trainer.train()

scheduler = get_linear_schedule_with_warmup(
    trainer.optimizer,
    num_warmup_steps=500,
    num_training_steps=trainer.args.num_train_epochs * len(df_train) // training_args.per_device_train_batch_size
)

# Save the model
trainer.save_model('./fine-tuned-bart-base')


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.045800,0.033868
1000,0.024100,0.025574
1500,0.021000,0.023700
2000,0.016300,0.021234
2500,0.014500,0.020416
3000,0.013000,0.020368
3500,0.013200,0.019948


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

In [32]:
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer
from datasets import Dataset, load_metric
import numpy as np
import torch

# Convert the validation DataFrame to a Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_test)

# Load the fine-tuned model and tokenizer
fine_tuned_model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')
fine_tuned_tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')

# Move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fine_tuned_model.to(device)

# Create a pipeline with the fine-tuned model
bart_model_fine_tuned = pipeline('text2text-generation', model=fine_tuned_model, tokenizer=fine_tuned_tokenizer, device=0)

# Tokenize the validation dataset using the same function as during training
def tokenize_function(examples):
    # Tokenize the inputs (disfluent text)
    model_inputs = fine_tuned_tokenizer(examples['disfluent'], max_length=512, truncation=True, padding="max_length")

    # Tokenize the labels (original text) using `text_target`
    labels = fine_tuned_tokenizer(text_target=examples['original'], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_test = dataset_test.map(tokenize_function, batched=True)

# Evaluate the model on the validation set
predictions = []
references = []

for example in tokenized_valid:
    # Generate output using the fine-tuned model
    output = bart_model_fine_tuned(example['disfluent'], max_length=50, min_length=5, do_sample=False)
    predictions.append(output[0]['generated_text'])
    references.append(example['original'])

# Calculate BLEU score
bleu_metric = load_metric("bleu")
bleu_score = bleu_metric.compute(predictions=[pred.split() for pred in predictions],
                                 references=[[ref.split()] for ref in references])
print(f"BLEU score: {bleu_score['bleu']}")

# Calculate Exact Match Accuracy
exact_match = np.mean([pred == ref for pred, ref in zip(predictions, references)])
print(f"Exact Match Accuracy: {exact_match * 100:.2f}%")


Map:   0%|          | 0/3643 [00:00<?, ? examples/s]

BLEU score: 0.887961983804101
Exact Match Accuracy: 68.49%


In [8]:
import pandas as pd
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback, get_linear_schedule_with_warmup
from datasets import Dataset

# Define model and tokenizer
model_name = 'facebook/bart-base'  # Changed to BART Base
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

# Convert the training and validation DataFrames to Hugging Face Datasets
dataset_train = Dataset.from_pandas(df_train)
dataset_valid = Dataset.from_pandas(df_valid)

# Define the tokenize_function
def tokenize_function(examples):
    # Tokenize the inputs (disfluent text)
    model_inputs = tokenizer(examples['disfluent'], max_length=512, truncation=True, padding="max_length")

    # Tokenize the labels (original text) using `text_target`
    labels = tokenizer(text_target=examples['original'], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the training and test datasets
tokenized_train = dataset_train.map(tokenize_function, batched=True)
tokenized_valid = dataset_valid.map(tokenize_function, batched=True)

# Define training arguments with regularization and learning rate scheduler
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,                    # Number of epochs
    per_device_train_batch_size=8,        # Increased batch size to balance performance
    per_device_eval_batch_size=8,
    warmup_steps=500,                      # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                     # L2 regularization (weight decay)
    logging_dir='./logs',
    logging_steps=200,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Implement Early Stopping Callback
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    callbacks=[early_stopping_callback]
)

# Training the model
trainer.train()

scheduler = get_linear_schedule_with_warmup(
    trainer.optimizer,
    num_warmup_steps=500,
    num_training_steps=trainer.args.num_train_epochs * len(df_train) // training_args.per_device_train_batch_size
)

# Save the model
trainer.save_model('./fine-tuned-bart-base')

Map:   0%|          | 0/7182 [00:00<?, ? examples/s]

Map:   0%|          | 0/3643 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,0.110100,0.009773
1000,0.007900,0.008805
1500,0.006800,0.008362
2000,0.004200,0.008638
2500,0.004400,0.008252
3000,0.003300,0.008953
3500,0.003200,0.008871
4000,0.002100,0.009274


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

In [11]:
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer
from datasets import Dataset, load_metric
import numpy as np
import torch

# Convert the validation DataFrame to a Hugging Face Dataset
dataset_test = Dataset.from_pandas(df_test)

# Load the fine-tuned model and tokenizer
fine_tuned_model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart-base')
fine_tuned_tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart-base')

# Move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
fine_tuned_model.to(device)

# Create a pipeline with the fine-tuned model
bart_model_fine_tuned = pipeline('text2text-generation', model=fine_tuned_model, tokenizer=fine_tuned_tokenizer, device=0)

# Tokenize the validation dataset using the same function as during training
def tokenize_function(examples):
    # Tokenize the inputs (disfluent text)
    model_inputs = fine_tuned_tokenizer(examples['disfluent'], max_length=512, truncation=True, padding="max_length")

    # Tokenize the labels (original text) using `text_target`
    labels = fine_tuned_tokenizer(text_target=examples['original'], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_test = dataset_test.map(tokenize_function, batched=True)

# Evaluate the model on the validation set
predictions = []
references = []

for example in tokenized_valid:
    # Generate output using the fine-tuned model
    output = bart_model_fine_tuned(example['disfluent'], max_length=50, min_length=5, do_sample=False)
    predictions.append(output[0]['generated_text'])
    references.append(example['original'])

# Calculate BLEU score
bleu_metric = load_metric("bleu")
bleu_score = bleu_metric.compute(predictions=[pred.split() for pred in predictions],
                                 references=[[ref.split()] for ref in references])
print(f"BLEU score: {bleu_score['bleu']}")

# Calculate Exact Match Accuracy
exact_match = np.mean([pred == ref for pred, ref in zip(predictions, references)])
print(f"Exact Match Accuracy: {exact_match * 100:.2f}%")


Map:   0%|          | 0/3643 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
<ipython-input-11-5d83241b6b63>:44: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("bleu")


The repository for bleu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bleu.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
BLEU score: 0.8820109066869678
Exact Match Accuracy: 67.33%
